Setup instructions for running pyspark in jupyter notebooks (using conda environment, such as "my-env"):
```
java -version # Should be 1.8.0_241 ... i.e. Final Java 8 release 
conda create -n my-env
conda activate -n my-env python=3.7 jupyter
conda install -c conda-forge pyspark
conda install -c anaconda ipykernel
python -m ipykernel install --user --name=my-env
jupyter notebook
```
Open new notebook file by selecting New > my-env

In [34]:
import numpy as np
import pandas as pd


from pyspark import SparkContext
from pyspark.sql import SparkSession

import pyspark.sql.functions as psf

First create a `SparkContext` which connects the Spark application to the cluster (in this case local).

In [2]:
sc = SparkContext('local[*]')
print(sc)
print(sc.version)

<SparkContext master=local[*] appName=pyspark-shell>
2.4.5


Then get or create a new `SparkSession` which we'll use to control the Spark driver.

In [6]:
spark = SparkSession.builder.getOrCreate()
print(spark)

List the contents of the Spark session using its `catalog` attribute, which has methods such as `listTables()`. (Currently nothing in the session.)

In [9]:
spark.catalog.listTables()

[]

Other useful `catalog` methods might include:
- `cacheTable(tableName)`/`uncacheTable(tableName)`/`isCached(tableName)`
- `createTable()`
- `listColumns(tableName)`
- `listFunctions(dbName=None)` -- functions registered in a specified database.

-------------------------------------

We can start off generating sequences to play with using the session:

In [14]:
spark.range(1000) # Returns dataframe

DataFrame[id: bigint]

In [15]:
spark.range(1000).toDF("number") # Give it column header 'number'

DataFrame[number: bigint]

We could assign that dataframe to a variable and work on that. Or we can keep chaining functions, using functional programming style, to minimise use of intermediate global objects and variables.

In [18]:
spark.range(1000)\
    .toDF("number")\
    .where("number % 2 = 0")\
    .count()

500

The executions we've run above are saved in the Spark application as previous "jobs". You can view the job history, among other things, by looking at the Spark UI, on port 4040 of the driver node, which in our local case is http://localhost:4040.

-----------------------

We can run SQL queries or dataframe manipulations through the `SparkSession`. But first we need to read some data into the session, using its `read` methods.

In [45]:
flights = spark.read.csv("/media/sf_M_DRIVE/datasets/demo/flight-delays/flights.csv", inferSchema=True, header=True)

In [21]:
flights.columns

['YEAR',
 'MONTH',
 'DAY',
 'DAY_OF_WEEK',
 'AIRLINE',
 'FLIGHT_NUMBER',
 'TAIL_NUMBER',
 'ORIGIN_AIRPORT',
 'DESTINATION_AIRPORT',
 'SCHEDULED_DEPARTURE',
 'DEPARTURE_TIME',
 'DEPARTURE_DELAY',
 'TAXI_OUT',
 'WHEELS_OFF',
 'SCHEDULED_TIME',
 'ELAPSED_TIME',
 'AIR_TIME',
 'DISTANCE',
 'WHEELS_ON',
 'TAXI_IN',
 'SCHEDULED_ARRIVAL',
 'ARRIVAL_TIME',
 'ARRIVAL_DELAY',
 'DIVERTED',
 'CANCELLED',
 'CANCELLATION_REASON',
 'AIR_SYSTEM_DELAY',
 'SECURITY_DELAY',
 'AIRLINE_DELAY',
 'LATE_AIRCRAFT_DELAY',
 'WEATHER_DELAY']

Here's a dataframe manipulation chain, finding the number of flights within each origin-destination group.

In [30]:
# from pyspark.sql.functions import desc

In [35]:
flights\
    .select('FLIGHT_NUMBER', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 
        'SCHEDULED_DEPARTURE', 'SCHEDULED_ARRIVAL',
        'DEPARTURE_TIME', 'ARRIVAL_TIME', 'ARRIVAL_DELAY')\
    .groupBy('ORIGIN_AIRPORT', 'DESTINATION_AIRPORT')\
    .count()\
    .withColumnRenamed('count', 'total_flights')\
    .sort(psf.desc('total_flights'))\
    .take(10) # For comparison with sql code, could be written as .limit(10)/.collect()

[Row(ORIGIN_AIRPORT='SFO', DESTINATION_AIRPORT='LAX', total_flights=13744),
 Row(ORIGIN_AIRPORT='LAX', DESTINATION_AIRPORT='SFO', total_flights=13457),
 Row(ORIGIN_AIRPORT='JFK', DESTINATION_AIRPORT='LAX', total_flights=12016),
 Row(ORIGIN_AIRPORT='LAX', DESTINATION_AIRPORT='JFK', total_flights=12015),
 Row(ORIGIN_AIRPORT='LAS', DESTINATION_AIRPORT='LAX', total_flights=9715),
 Row(ORIGIN_AIRPORT='LGA', DESTINATION_AIRPORT='ORD', total_flights=9639),
 Row(ORIGIN_AIRPORT='LAX', DESTINATION_AIRPORT='LAS', total_flights=9594),
 Row(ORIGIN_AIRPORT='ORD', DESTINATION_AIRPORT='LGA', total_flights=9575),
 Row(ORIGIN_AIRPORT='SFO', DESTINATION_AIRPORT='JFK', total_flights=8440),
 Row(ORIGIN_AIRPORT='JFK', DESTINATION_AIRPORT='SFO', total_flights=8437)]

The equivalent can be written as a SQL query, and from Spark's point of view they are an identical implementation. To run SQL queries we must first convert the dataframe into a database table. We then query the table by passing SQL code to the session driver.

In [26]:
flights.createOrReplaceTempView("flights")

In [33]:
spark.sql("""
    FROM flights
        SELECT origin_airport, destination_airport, count(*) AS total_flights
        GROUP BY origin_airport, destination_airport
        ORDER BY total_flights DESC
        LIMIT 10
    """).collect()

[Row(origin_airport='SFO', destination_airport='LAX', total_flights=13744),
 Row(origin_airport='LAX', destination_airport='SFO', total_flights=13457),
 Row(origin_airport='JFK', destination_airport='LAX', total_flights=12016),
 Row(origin_airport='LAX', destination_airport='JFK', total_flights=12015),
 Row(origin_airport='LAS', destination_airport='LAX', total_flights=9715),
 Row(origin_airport='LGA', destination_airport='ORD', total_flights=9639),
 Row(origin_airport='LAX', destination_airport='LAS', total_flights=9594),
 Row(origin_airport='ORD', destination_airport='LGA', total_flights=9575),
 Row(origin_airport='SFO', destination_airport='JFK', total_flights=8440),
 Row(origin_airport='JFK', destination_airport='SFO', total_flights=8437)]

We can take the queried data using either SQL or DataFrame styles, and if we create a local `pandas` copy using `.toPandas()`

In [54]:
flights\
    .select('FLIGHT_NUMBER', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 
        'SCHEDULED_DEPARTURE', 'SCHEDULED_ARRIVAL',
        'DEPARTURE_TIME', 'ARRIVAL_TIME', 'ARRIVAL_DELAY')\
    .groupBy('ORIGIN_AIRPORT', 'DESTINATION_AIRPORT')\
    .mean('ARRIVAL_DELAY')\
    .withColumnRenamed('avg(ARRIVAL_DELAY)', 'avg_delay')\
    .sort(desc('avg_delay'))\
    .limit(20)\
    .toPandas()

,ORIGIN_AIRPORT,DESTINATION_AIRPORT,avg_delay
0,IAD,TTN,381.000000
1,SWF,PBI,260.500000
2,RIC,CAE,228.000000
3,RDU,IND,208.000000
4,10581,11618,163.000000
5,FCA,MSO,148.000000
6,SWF,RSW,140.000000
7,10581,12953,138.333333
8,14843,12264,122.000000
9,OAK,FLL,106.000000


-------------------------------------

In [61]:
spark.catalog

AnalysisException: "Table 'flights' does not exist in database 'default'.;"

In [5]:
my_range = sc.parallelize(range(0,int(5e5)))

In [6]:
my_range

PythonRDD[1] at RDD at PythonRDD.scala:53

In [7]:
my_range.count()

500000

In [11]:
my_range.filter(lambda a: a % 2 == 0).count()

250000

In [16]:
my_range\
    .filter(lambda a: a % 2 == 0)\
    .map(lambda a: a ** 2)\
    .mean()

83332833334.0

In [20]:
my_range\
    .filter(lambda a: a % 2 == 0)\
    .map(lambda a: a ** 2)\
    .reduce(mean)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 6.0 failed 1 times, most recent failure: Lost task 1.0 in stage 6.0 (TID 49, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/sean/anaconda3/envs/spark-play/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 377, in main
    process()
  File "/home/sean/anaconda3/envs/spark-play/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 372, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/home/sean/anaconda3/envs/spark-play/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 400, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/home/sean/anaconda3/envs/spark-play/lib/python3